In [1]:
# Import libraries
import pandas as pd
import statsmodels as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
# U P L O A D
datafile = "/Users/justinpassek/Documents/6469_Caesars_Dynamic_Elements_20181030_055039_2000227069.csv"
df = pd.read_csv(datafile,skiprows=81)
print(df)

        Advertiser                 Dynamic Profile  \
0          Caesars  CET_2018_Dubai_Dynamic_160x600   
1          Caesars  CET_2018_Dubai_Dynamic_160x600   
2          Caesars  CET_2018_Dubai_Dynamic_160x600   
3          Caesars  CET_2018_Dubai_Dynamic_160x600   
4          Caesars  CET_2018_Dubai_Dynamic_160x600   
5          Caesars  CET_2018_Dubai_Dynamic_160x600   
6          Caesars  CET_2018_Dubai_Dynamic_160x600   
7          Caesars  CET_2018_Dubai_Dynamic_160x600   
8          Caesars  CET_2018_Dubai_Dynamic_160x600   
9          Caesars  CET_2018_Dubai_Dynamic_160x600   
10         Caesars  CET_2018_Dubai_Dynamic_160x600   
11         Caesars  CET_2018_Dubai_Dynamic_160x600   
12         Caesars  CET_2018_Dubai_Dynamic_160x600   
13         Caesars  CET_2018_Dubai_Dynamic_160x600   
14         Caesars  CET_2018_Dubai_Dynamic_160x600   
15         Caesars  CET_2018_Dubai_Dynamic_160x600   
16         Caesars  CET_2018_Dubai_Dynamic_160x600   
17         Caesars  CET_2018

In [3]:
# M U T A T E

# change column names
new_names = {'Mercury Res Module - Sales : RES Step 5 (Confirmation Page): Total Conversions':'Conversions',
            'Feed 1 - Reporting dimension 2 value':'CTA',
            'Feed 1 - Reporting dimension 1 value':'Image'}
df.rename(columns=new_names,inplace=True)

# add conversion rate column
df['ConvRate']=df.Conversions / df.Impressions
print(df)

        Advertiser                 Dynamic Profile  \
0          Caesars  CET_2018_Dubai_Dynamic_160x600   
1          Caesars  CET_2018_Dubai_Dynamic_160x600   
2          Caesars  CET_2018_Dubai_Dynamic_160x600   
3          Caesars  CET_2018_Dubai_Dynamic_160x600   
4          Caesars  CET_2018_Dubai_Dynamic_160x600   
5          Caesars  CET_2018_Dubai_Dynamic_160x600   
6          Caesars  CET_2018_Dubai_Dynamic_160x600   
7          Caesars  CET_2018_Dubai_Dynamic_160x600   
8          Caesars  CET_2018_Dubai_Dynamic_160x600   
9          Caesars  CET_2018_Dubai_Dynamic_160x600   
10         Caesars  CET_2018_Dubai_Dynamic_160x600   
11         Caesars  CET_2018_Dubai_Dynamic_160x600   
12         Caesars  CET_2018_Dubai_Dynamic_160x600   
13         Caesars  CET_2018_Dubai_Dynamic_160x600   
14         Caesars  CET_2018_Dubai_Dynamic_160x600   
15         Caesars  CET_2018_Dubai_Dynamic_160x600   
16         Caesars  CET_2018_Dubai_Dynamic_160x600   
17         Caesars  CET_2018

In [4]:
# C L E A N

# 1000 impression minimum
df = df[df["Impressions"]>5000]

# rm unidentifiable images and cta
df = df[df.Image != '(not set)']
df = df[df.CTA != '(not set)']
df = df[df.Image != 'Test Headline']
df = df[df.CTA != 'Test Headline']
df = df[df.Image != '---']
df = df[df.CTA != '---']

# rm nas
df = df.dropna(subset=["CTA", "Image"])

# narrow to relevant columns
imp_columns = ["Image","CTA","ConvRate","Impressions","Conversions"]
df = df[imp_columns]

print(df)

     Image                     CTA  ConvRate  Impressions  Conversions
3085   ATL       FALL GETAWAY SALE  0.001355        20663         28.0
3086   ATL       FALL GETAWAY SALE  0.001637         5497          9.0
3087   ATL       FALL GETAWAY SALE  0.000510         5879          3.0
3088   ATL       FALL GETAWAY SALE  0.001528         7199         11.0
3112   ATL       FALL GETAWAY SALE  0.002039         6376         13.0
3118   ATL  Book Early<br>And Save  0.000000         5991          0.0
3119   ATL  Book Early<br>And Save  0.000075        26504          2.0
3120   ATL  Book Early<br>And Save  0.000152         6585          1.0
3121   ATL  Book Early<br>And Save  0.000315         9512          3.0
3123   BAC  Book Early<br>And Save  0.000245        24510          6.0
3124   BAC  Book Early<br>And Save  0.000000         5967          0.0
3125   BAC  Book Early<br>And Save  0.000109         9145          1.0
3127   CAC  Book Early<br>And Save  0.000283        21178          6.0
3128  

In [5]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

pvdf = df.groupby(['Image','CTA'],as_index=False)[('Impressions','Conversions')].sum()
pvdf['ConvRate'] = pvdf.Conversions/pvdf.Impressions
print(pvdf)

pvdf_v0 = df.groupby(['Image'],as_index=False)[('Impressions','Conversions')].sum()
pvdf_v0['ConvRate'] = pvdf_v0.Conversions/pvdf_v0.Impressions
print(pvdf_v0)

pvdf_v1 = df.groupby(['CTA'],as_index=False)[('Impressions','Conversions')].sum()
pvdf_v1['ConvRate'] = pvdf_v1.Conversions/pvdf_v1.Impressions
print(pvdf_v1)

  Image                     CTA  Impressions  Conversions  ConvRate
0   ATL  Book Early<br>And Save        48592          6.0  0.000123
1   ATL       FALL GETAWAY SALE      1795061        833.0  0.000464
2   BAC  Book Early<br>And Save        39622          7.0  0.000177
3   BAC       FALL GETAWAY SALE       742295        205.0  0.000276
4   BAC            LOCALS OFFER       106614         34.0  0.000319
5   CAC  Book Early<br>And Save        34549         10.0  0.000289
6   CAC       FALL GETAWAY SALE       782069        195.0  0.000249
  Image  Impressions  Conversions  ConvRate
0   ATL      1843653        839.0  0.000455
1   BAC       888531        246.0  0.000277
2   CAC       816618        205.0  0.000251
                      CTA  Impressions  Conversions  ConvRate
0  Book Early<br>And Save       122763         23.0  0.000187
1       FALL GETAWAY SALE      3319425       1233.0  0.000371
2            LOCALS OFFER       106614         34.0  0.000319


In [6]:
# anova test
ConvRate_Total = sum(df.Conversions)/sum(df.Impressions)
print(ConvRate_Total)

formula = 'ConvRate ~ C(Image) + C(CTA) + C(Image):C(CTA)'
model = ols(formula,data=df).fit()
aov_table = anova_lm(model, typ=1)
print(aov_table)

0.0003635029511367498
                    df    sum_sq       mean_sq         F    PR(>F)
C(Image)           2.0  0.000003  1.374951e-06  4.187922  0.017576
C(CTA)             2.0  0.000002  8.032472e-07  2.446587  0.091124
C(Image):C(CTA)    4.0  0.000002  4.276885e-07  1.302684  0.273282
Residual         114.0  0.000037  3.283134e-07       NaN       NaN


In [7]:
p_v0 = aov_table.iloc[0,4]
p_v1 = aov_table.iloc[1,4]

if p_v0 < .05:
    print(aov_table.index.values[0], 'Result IS significant')
else:
    print(aov_table.index.values[0], 'Result IS NOT significant')

if p_v1 < .05:
    print(aov_table.index.values[1], 'Result IS significant')
else:
    print(aov_table.index.values[1], 'Result IS NOT significant')

C(Image) Result IS significant
C(CTA) Result IS NOT significant


In [71]:
# Manual Anova

# Degrees of Freedom
N = len(df.ConvRate)
print("N is %s" % N)

df_a = len(df.Image.unique()) - 1
print("df_a is %s" % df_a)

df_b = len(df.CTA.unique()) - 1
print("df_b is %s" % df_b)

df_axb = df_a*df_b
print("df_axb is %s" % df_axb)

df_w = N - (len(df.Image.unique())*len(df.CTA.unique()))
print("df_w is %s" % df_w)

N is 197
df_a is 5
df_b is 5
df_axb is 25
df_w is 161


In [58]:
#Sum of Squares
grand_mean = df['ConvRate'].mean()
print("Grand mean is %s" % grand_mean)

ssq_a = sum([(df[df.Image ==l].ConvRate.mean()-grand_mean)**2 for l in df.Image])
print("Sum of Squares Image is %s" % ssq_a)

ssq_b = sum([(df[df.CTA ==l].ConvRate.mean()-grand_mean)**2 for l in df.CTA])
print("Sum of Squares CTA is %s" % ssq_b)

ssq_t = sum((df.ConvRate - grand_mean)**2)
print("Sum of Squares Total is %s" % ssq_t)

Grand mean is 0.001384318480535868
Sum of Squares Image is 0.0013541743125461444
Sum of Squares CTA is 0.001577351332513837
Sum of Squares Total is 0.003793301992259968


In [63]:
ATL = df[df.Image == 'ATL']
print(ATL)

BAC = df[df.Image == 'BAC']
print(BAC)

FLV = df[df.Image == 'FLV']
print(FLV)

CLV = df[df.Image == 'CLV']
print(CLV)

ATL_CTA_means = [ATL[ATL.CTA == d].ConvRate.mean() for d in ATL.CTA]
print(ATL_CTA_means)

BAC_CTA_means = [BAC[BAC.CTA == d].ConvRate.mean() for d in BAC.CTA]
print(BAC_CTA_means)

FLV_CTA_means = [FLV[FLV.CTA == d].ConvRate.mean() for d in FLV.CTA]
print(BAC_CTA_means)

CLV_CTA_means = [CLV[CLV.CTA == d].ConvRate.mean() for d in CLV.CTA]
print(CLV_CTA_means)

ssq_w = sum((ATL.ConvRate - ATL_CTA_means)**2) +sum((BAC.ConvRate - BAC_CTA_means)**2) +sum((FLV.ConvRate - FLV_CTA_means)**2) +sum((CLV.ConvRate - CLV_CTA_means)**2)
print(ssq_w)

     Image                     CTA  ConvRate
3085   ATL       FALL GETAWAY SALE  0.001355
3086   ATL       FALL GETAWAY SALE  0.001637
3087   ATL       FALL GETAWAY SALE  0.000510
3088   ATL       FALL GETAWAY SALE  0.001528
3100   ATL       FALL GETAWAY SALE  0.000000
3103   ATL       FALL GETAWAY SALE  0.000000
3106   ATL       FALL GETAWAY SALE  0.000000
3107   ATL       FALL GETAWAY SALE  0.000623
3108   ATL       FALL GETAWAY SALE  0.002155
3109   ATL       FALL GETAWAY SALE  0.000921
3110   ATL       FALL GETAWAY SALE  0.002706
3111   ATL       FALL GETAWAY SALE  0.000651
3112   ATL       FALL GETAWAY SALE  0.002039
3113   ATL       FALL GETAWAY SALE  0.001761
3118   ATL  Book Early<br>And Save  0.000000
3119   ATL  Book Early<br>And Save  0.000075
3120   ATL  Book Early<br>And Save  0.000152
3121   ATL  Book Early<br>And Save  0.000315
3130   ATL       FALL GETAWAY SALE  0.000413
3131   ATL       FALL GETAWAY SALE  0.000279
3132   ATL       FALL GETAWAY SALE  0.000173
3133   ATL

In [64]:
# Sum of Squares interaction
ssq_axb = ssq_t-ssq_a-ssq_b-ssq_w
print(ssq_axb)

-6.299507209109388e-05


In [65]:
# Mean squares
ms_a = ssq_a/df_a
print(ms_a)

ms_b = ssq_b/df_b
print(ms_b)

ms_axb = ssq_axb/df_axb
print(ms_axb)

ms_w = ssq_w/df_w
print(ms_w)

0.0002708348625092289
0.00031547026650276743
-2.519802883643755e-06
5.74392185895081e-06


In [66]:
# F-statistic
f_a = ms_a/ms_w
print(f_a)

f_b = ms_b/ms_w
print(f_b)

f_axb = ms_axb/ms_w
print(f_axb)

47.151557622112186
54.922450940234675
-0.43869031395632957


In [67]:
# P-values
p_a = stats.f.sf(f_a, df_a, df_w)
print(p_a)

p_b = stats.f.sf(f_b, df_b, df_w)
print(p_b)

p_axb = stats.f.sf(f_axb, df_axb, df_w)
print(p_axb)

7.579341233167899e-30
4.478406534244682e-33
1.0


In [68]:
# Table
results = {'sum_sq':[ssq_a, ssq_b, ssq_axb, ssq_w],
           'df':[df_a, df_b, df_axb, df_w],
           'F':[f_a, f_b, f_axb, 'NaN'],
            'PR(>F)':[p_a, p_b, p_axb, 'NaN']}
columns=['sum_sq', 'df', 'F', 'PR(>F)']
 
aov_table1 = pd.DataFrame(results, columns=columns,
                          index=['Image', 'CTA', 
                          'Image:CTA', 'Residual'])

print(aov_table1)

             sum_sq   df        F       PR(>F)
Image      0.001354    5  47.1516  7.57934e-30
CTA        0.001577    5  54.9225  4.47841e-33
Image:CTA -0.000063   25 -0.43869            1
Residual   0.000925  161      NaN          NaN


In [69]:
# Table with effect sizes
def eta_squared(aov):
    aov['eta_sq'] = 'NaN'
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    return aov

def omega_squared(aov):
    mse = aov['sum_sq'][-1]/aov['df'][-1]
    aov['omega_sq'] = 'NaN'
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)
    return aov


eta_squared(aov_table1)
omega_squared(aov_table1)
print(aov_table1)

             sum_sq   df        F       PR(>F)    eta_sq  omega_sq
Image      0.001354    5  47.1516  7.57934e-30  0.356991  0.348891
CTA        0.001577    5  54.9225  4.47841e-33  0.415825  0.407637
Image:CTA -0.000063   25 -0.43869            1 -0.016607 -0.054380
Residual   0.000925  161      NaN          NaN       NaN       NaN
